# Lab 5 

## Cargando y Revisando los datos

In [150]:
import pandas as pd 
import numpy as np 

In [151]:
df = pd.read_csv('Admission_Predict.csv')

In [152]:
df.head(5)

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In [153]:
df.isnull().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

Con esta data como viene podemos empezar a trabajar sin embargo tenemos problemas en las variables X que estoy identificando porque usan diferente escala, por lo tanto las debemos de organizar. 
Las variables que usarare de X serán GRE Score, TOEFL SCORE, cgpa y Research. ya que las demas son situacionales y personales y quiero trabajar con datos empíricos. Por lo tanto tambien nos quedaremos con su SOP. 

In [154]:
df['cons'] = 1

In [155]:
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit ', 'cons'],
      dtype='object')

In [156]:
X = df[['cons','GRE Score' , 'TOEFL Score', 'CGPA','SOP','Research' ]].to_numpy()

In [157]:
y = df['Chance of Admit '].to_numpy()

In [158]:
print(y.shape,X.shape)

(400,) (400, 6)


Para hacer que nuestras variables funcionen en la misma escala haremos el proceso de StandarScalet de Sklearn

In [159]:
from sklearn.preprocessing import StandardScaler

In [160]:
scaler = StandardScaler()

In [161]:
X_S = scaler.fit_transform(X) 

Ahora ya tenemos nuestras variables estandarizadas ya podemos cargar nuestras funciones y empezar a hacer pruebas

## Cargando Funciones con regularizador

In [162]:
def linear_cost(X, y, theta): 
    h = X @ theta
    return ((y-h)**2).sum() / (2 * len(X)) 

In [163]:
def linear_cost_gradient(X, y, theta, lambda_p = 10): 
    h = X @ theta
    lambda_f = (lambda_p/(2*len(X))) * (theta**2).sum()
    return (X.T @ (h-y))/ len(X) + lambda_f

In [164]:
def gradient_descent(
X,y,theta_0,linear_cost,linear_cost_gradient,
    learning_rate = 0.0001, threshold=0.001, max_iter=10000,lambda_p = 10): 
    
    theta = theta_0
    iteration = 0 
    costs = []
    thetas = []
    
    while np.linalg.norm(linear_cost_gradient(X,y,theta)) > threshold and iteration < max_iter:
        iteration += 1 
        theta = theta - (learning_rate * linear_cost_gradient(X,y,theta) + lambda_p * theta)
        costs.append(linear_cost(X,y,theta))
        thetas.append(theta.copy())
        
    return theta, costs, thetas

# Polinomio Grado 1

Para empezar con el ejercicio probaremos hacer un polinomio de grado 1 que es con la data como la tenemos actualemente

### Thetas Iniciales y Separación de data

In [165]:
m, n = X.shape
theta_0 = np.random.rand(n,1)
print(theta_0.shape)

(6, 1)


In [166]:
np.random.shuffle(X_S)
np.random.shuffle(y)

In [167]:
X_train = X_S[:200]
X_cv = X_S[201:300]
X_test = X_S[301:400]
print(X_train.shape,X_cv.shape,X_test.shape)

(200, 6) (99, 6) (99, 6)


Decidi hacer una partición de 50% para el training set y 25% y 25% respectivamente para los test debido a la poca cantidad de datos

In [168]:
y_train = y[:200]
y_cv = y[201:300]
y_test = y[301:400]
print(y_train.shape,y_cv.shape,y_test.shape)

(200,) (99,) (99,)


In [69]:
theta, costs, thetas = gradient_descent(X_train,y_train,theta_0,linear_cost,linear_cost_gradient,0.00001,0.001,10000,0)

In [169]:
print(costs[len(costs)-1])

83.67775226047816


Con un regularizador de Grado 0 Obtenemos este resultado en el costo. Cree que si aumentamos el regulador a  obtenemos un mejor resultado, veremos a continuación

In [170]:
theta_1, costs_1, thetas_1 = gradient_descent(X_train,y_train,theta_0,linear_cost,linear_cost_gradient,0.00001,0.001,10000,1)

In [171]:
print(costs_1[len(costs_1)-1])

54.6783457395602


Si colocamos un regularizador de dos encontramos que ya tenemos un costo  bajo. pero no lo suficiente para considerar que tiene motivos para hacer las otras pruebas. por lo tanto pasaremos a el polinomio de grado 2

## Polinomio Grado 2

In [116]:
df['CGPA_2'] = df['CGPA'] ** 2

Usaremos de variable el CGPA ya que considero que es una de las variables más importantes del ejercicio 

In [92]:
X2 = df[['cons','GRE Score' , 'TOEFL Score', 'CGPA','SOP','CGPA_2','Research' ]].to_numpy()
y2 = df['Chance of Admit '].to_numpy()

In [119]:
print(y2.shape,X2.shape)

(400,) (400, 7)


In [94]:
X_S_2 = scaler.fit_transform(X2) 

### Thetas Iniciales y Separación de data

m, n = X2.shape
theta_0_2 = np.random.rand(n,1)
print(theta_0.shape)

In [97]:
np.random.shuffle(X_S_2)
np.random.shuffle(y2)

In [98]:
X2_train = X_S_2[:200]
X2_cv = X_S_2[201:300]
X2_test = X_S_2[301:400]
print(X2_train.shape,X2_cv.shape,X2_test.shape)

(200, 7) (99, 7) (99, 7)


In [99]:
y2_train = y2[:200]
y2_cv = y2[201:300]
y2_test = y2[301:400]
print(y2_train.shape,y2_cv.shape,y2_test.shape)

(200,) (99,) (99,)


In [100]:
theta2, costs2, thetas2 = gradient_descent(X2_train,y2_train,theta_0_2,linear_cost,linear_cost_gradient,0.00001,0.001,10000,0)

In [101]:
print(costs2[len(costs2)-1])

68.25464306374033


In [112]:
theta2_1, costs2_1, thetas2_1 = gradient_descent(X2_train,y2_train,theta_0_2,linear_cost,linear_cost_gradient,0.00001,0.001,10000,1)

In [113]:
print(costs2_1[len(costs2_1)-1])

54.34159061469193


En este polinomio logramos reducir un poco el costo al colocar un regularizador de 1. pero todavia no lo suficiente. Probaremos con un polinomio de grado 3

### Polinomio Grado 3

In [117]:
df['CGPA_3'] = df['CGPA'] ** 3

In [118]:
X3 = df[['cons','GRE Score' , 'TOEFL Score', 'CGPA','SOP','CGPA_2','CGPA_3','Research' ]].to_numpy()
y3 = df['Chance of Admit '].to_numpy()

In [120]:
print(y3.shape,X3.shape)

(400,) (400, 8)


In [121]:
X_S_3 = scaler.fit_transform(X3) 

### Thetas Iniciales y Separación de data

In [123]:
m, n = X3.shape
theta_0_3 = np.random.rand(n,1)
print(theta_0_3.shape)

(8, 1)


In [124]:
np.random.shuffle(X_S_3)
np.random.shuffle(y3)

In [125]:
X3_train = X_S_3[:200]
X3_cv = X_S_3[201:300]
X3_test = X_S_3[301:400]
print(X3_train.shape,X3_cv.shape,X3_test.shape)

(200, 8) (99, 8) (99, 8)


In [126]:
y3_train = y3[:200]
y3_cv = y3[201:300]
y3_test = y3[301:400]
print(y3_train.shape,y3_cv.shape,y3_test.shape)

(200,) (99,) (99,)


In [130]:
theta3, costs3, thetas3 = gradient_descent(X3_train,y3_train,theta_0_3,linear_cost,linear_cost_gradient,0.00001,0.001,10000,0)

In [131]:
print(costs3[len(costs3)-1])

66.57883336496948


In [136]:
theta3_1, costs3_1, thetas3_1 = gradient_descent(X3_train,y3_train,theta_0_3,linear_cost,linear_cost_gradient,0.000001,0.001,10000,1)

In [137]:
print(costs3_1[len(costs3_1)-1])

53.56989897838429


Como podemos ver en el siguiente codigo logramos reducir el costo de un polinomio grado 3 a ser mas pequeño que los de polinomio grado 1 y 2. por lo tango nos dentendremos aca y haremos las pruebas de cross validation y test para este polinomio

### CV polinomio grado 3 con regularizador de 1

# Lo entregare así que tengo duda de porque no me deja multiplicarlos. No entiendo el resultado que me da las thetas que me da un array mas grande del que necesito. 